In [26]:
import pandas as pd
import numpy as np
from datetime import datetime
import pymysql
import warnings
warnings.filterwarnings('ignore')
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [27]:
sql = '''
select
tprm.order_create_time, tprm.all_deposit 总押金, tprm.all_rental 总租金, tprm.purchase_amount 采购金额, tod.product_name, tod.commodity_premium 溢价费
,om.order_method, om.order_type, cc.channel_type_id, cc.name 来源渠道, tod.sku_attributes, tod.new_actual_money 买断金额, tprm.total_receivable
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order_details tod on tod.order_id=tprm.order_id
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_channel cc on om.channel = cc.scene 
where tprm.purchase_amount>0
and date_format(tprm.order_create_time, '%Y-%m-%d')>='2024-12-01'
and date_format(tprm.order_create_time, '%Y-%m-%d')<='2025-06-30'
'''
df = clean.query(sql)

In [28]:
def qudao_type(a, c, d, e):
    a = str(a)
    if (d in [2, 3, 80] and e=='ZFB_ORDER')  or c == 1:
        return "芝麻租物"
    elif e=='DY_ORDER':
        return "抖音渠道"
    elif d in [48, 81] and e=='ZFB_ORDER' and '灯火联投测试' not in a:
        return "搜索渠道"
    elif e=='JD_ORDER':
        return '京东渠道'
    else:
        return a
def getrecanzhi(s):
    if s["租赁方案"]=="租完即送":
        return  0
    elif s["残值机型"]=="苹果16":
        return s["采购金额"]*0.6
    elif s["残值机型"]=="苹果15":
        return s["采购金额"]*0.47
    elif s["残值机型"]=="苹果14":
        return s["采购金额"]*0.4
    elif s["残值机型"]=="苹果14以下":
        return s["采购金额"]*0.375
    else:
        return s["采购金额"]*0.425

In [29]:
df.loc[:, '下单日期'] = df.order_create_time.dt.strftime('%Y-%m-%d')
df.loc[:, '下单月份'] = df.order_create_time.dt.strftime('%Y-%m')
df.loc[:, '来源渠道'] = df.来源渠道.fillna('未知渠道')
df.loc[:, '归属渠道'] = df.apply(lambda x: qudao_type(x['来源渠道'], x['order_method'], x['channel_type_id'], x['order_type']), axis=1)
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:, '残值机型'] = np.where(df["product_name"].str.contains('iPhone'),  
                                np.where(df["product_name"].str.contains('16'), "苹果16",
                                        np.where(df['product_name'].str.contains('15'), "苹果15",
                                                np.where(df['product_name'].str.contains('14'), "苹果14", '苹果14以下'))),"非苹果")
df.loc[:, '残值'] = df.apply(getrecanzhi, axis=1)
df.loc[:, '总应收'] = df.总租金+df.残值+df.溢价费
df.loc[:, '残值估算毛利'] = np.where(df.下单月份<='2024-09', df.总应收 - df.采购金额, df.total_receivable - df.采购金额)
df.loc[:, '买断估算毛利'] = df.买断金额 - df.采购金额 + df.溢价费
def data_group(df):
        df_group = df.groupby('下单月份').agg({'总押金': 'mean', '总租金': 'mean', '采购金额': 'mean', '残值估算毛利': 'mean', '买断估算毛利': 'mean'}).rename(columns={'总租金': '总租金均值', '总押金': '总押金均值', '采购金额': '采购成本均值'})
        df_group.loc[:, '租售比'] = (df_group.总租金均值/df_group.总押金均值).map(lambda x: format(x,".2%"))
        df_group.loc[:, '买断估算毛利率'] = (df_group.买断估算毛利/df_group.采购成本均值).map(lambda x: format(x,".2%"))
        df_group.loc[:, '残值估算毛利率'] = (df_group.残值估算毛利/df_group.采购成本均值).map(lambda x: format(x,".2%"))
        return df_group
df_group_all = data_group(df)
df_zm = df[df.归属渠道=='芝麻租物']
df_group_zm = data_group(df_zm)
df_ss = df[df.归属渠道=='搜索渠道']
df_group_ss = data_group(df_ss)
today = datetime.now().strftime('%Y%m%d%H')
with pd.ExcelWriter(f'F:/需求/瑞彤需求/租售比_{today}.xlsx', engine='openpyxl') as writer:
        df_group_all.to_excel(writer, sheet_name='总体租售比')
        df_group_zm.to_excel(writer, sheet_name='芝麻租物租售比')
        df_group_ss.to_excel(writer, sheet_name='搜索渠道租售比')